# Code that reproduces the complete analysis from Rocklin et al., 2017, Science

In [1]:
# Import `Python` modules
import os
import sys
import pandas
import scipy.stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.5)

In [2]:
# Define input variables
data_dir = 'data/Rocklin_2017_Science/'
designed_sequences_file = os.path.join(data_dir, 'designed_protein_sequences.csv')
experimental_summary_file = os.path.join(data_dir, 'experimental_summary.csv')
fastq_dir = '/work/05402/haddox/jupyter/sd2e-community/archive/ingest/Q0/sd2.biofab.upload/Rocklin_ProtStab/'
pear_path = '/home/05402/haddox/software/pear/bin/pear'
output_dir = 'results/'

Call the python script

In [ ]:
# Write the command to carry everything out
cmd = ' '.join([
    'python',
    'scripts/compute_ec50_values_from_deep_sequencing_data.py',
    '--designed_sequences_file {0}'.format(designed_sequences_file),
    '--experimental_summary_file {0}'.format(experimental_summary_file),
    '--fastq_dir {0}'.format(fastq_dir),
    '--pear_path {0}'.format(pear_path),
    '--output_dir {0}'.format(output_dir)
])

! {cmd}

Making the directory: results/
Making the directory: results/paired_FASTQ_files
Making the directory: results/counts
Making the directory: results/ec50_values

Aggergating paired-end FASTQ files for the trypsin-treated sample and for selection index 0
Here is a list of R1 files: /work/05402/haddox/jupyter/sd2e-community/archive/ingest/Q0/sd2.biofab.upload/Rocklin_ProtStab/gabe1_S1_R1_001.fastq
Here is a list of R2 files: /work/05402/haddox/jupyter/sd2e-community/archive/ingest/Q0/sd2.biofab.upload/Rocklin_ProtStab/gabe1_S1_R1_001.fastq

Assembling the files /work/05402/haddox/jupyter/sd2e-community/archive/ingest/Q0/sd2.biofab.upload/Rocklin_ProtStab/gabe1_S1_R1_001.fastq and /work/05402/haddox/jupyter/sd2e-community/archive/ingest/Q0/sd2.biofab.upload/Rocklin_ProtStab/gabe1_S1_R2_001.fastq, outputting the assembled reads to a file called results/paired_FASTQ_files/trypsin_0-1.assembled.fastq

Using PARE to assembling the files: /work/05402/haddox/jupyter/sd2e-community/archive/ingest/

## Make sure the protein counts generated by the pipeline match the original protein counts reported by the Rocklin et al. study

Read in names and sequences of all input designs

In [ ]:
designed_seqs_df = pandas.read_csv(designed_sequences_file)
designed_seqs_df.set_index('name', inplace=True)
designed_seqs_df

Read in counts from the above pipeline

In [ ]:
proteases = ['chymotrypsin', 'trypsin']
counts_dfs = {
    'pipeline' : {},
    'original' : {}
}
counts_dir = os.path.join(output_dir, 'counts/')

for protease in proteases:
    
    # Read counts into dataframe
    df = pandas.read_csv(os.path.join(counts_dir, '{0}.counts'.format(protease)), sep=' ')
    df.set_index('name', inplace=True)
    print("Number of sequences for {0} with zero starting counts : {1}".format(
        protease,
        sum(df['counts0'] == 0)
    ))    
    counts_dfs['pipeline'][protease] = df
    
counts_dfs['pipeline']['trypsin']

Read in counts from the initial study

In [ ]:
# Read in the counts from the original study
for protease in proteases:
    
    # Read counts into dataframe
    if protease == 'chymotrypsin':
        df = pandas.read_csv('data/original_Rocklin_counts/rd4_chymo.counts', sep=' ')
    elif protease == 'trypsin':
        df = pandas.read_csv('data/original_Rocklin_counts/rd4_tryp.counts', sep=' ')
    else:
        raise ValueError()
    df.set_index('name', inplace=True)
    print("Number of sequences for {0} with zero starting counts : {1}".format(
        protease,
        sum(df['counts0'] == 0)
    ))
    
    counts_dfs['original'][protease] = df
    
counts_dfs['original']['chymotrypsin']
# counts_dfs['original']['trypsin']

Count the number of sequences in each dataset

In [ ]:
# Get a list of sequence names in each dataset
original_trypsin_seqs = list(counts_dfs['original']['trypsin'].index.values)
pipeline_trypsin_seqs = list(counts_dfs['pipeline']['trypsin'].index.values)
original_chymotrypsin_seqs = list(counts_dfs['original']['chymotrypsin'].index.values)
pipeline_chymotrypsin_seqs = list(counts_dfs['pipeline']['chymotrypsin'].index.values)
input_seqs = designed_seqs_df.index.values

# Check that all sequences are unique
for seqs in [
    original_trypsin_seqs, pipeline_trypsin_seqs, original_chymotrypsin_seqs,
    pipeline_chymotrypsin_seqs, input_seqs
]:
    assert(len(seqs) == len(set(seqs)))

# Print the results
print("Number of input sequences is: {0}".format(len(input_seqs)))
print("Number of sequences with counts from the original study for the two proteases is: {0} and {1}".format(
    len(original_trypsin_seqs), len(original_chymotrypsin_seqs)
))
print("Number of sequences with counts from the pipeline for the two proteases is: {0} and {1}".format(
    len(pipeline_trypsin_seqs), len(pipeline_chymotrypsin_seqs)
))

Quantify the number of starting sequences with counts data in each dataset

In [ ]:
print(len(set.intersection(set(input_seqs), set(original_trypsin_seqs))))
print(len(set.intersection(set(input_seqs), set(pipeline_trypsin_seqs))))

print(len(set.intersection(set(input_seqs), set(original_chymotrypsin_seqs))))
print(len(set.intersection(set(input_seqs), set(pipeline_chymotrypsin_seqs))))

In [ ]:
# Report the number of input sequences and the number of sequences with counts data for
# each protease and for the original Rocklin data vs. the pipeline data

# Quantify the number of sequences that are overlapping between the two datasets
overlapping_trypsin_seqs = set.intersection(set(original_trypsin_seqs), set(pipeline_trypsin_seqs))
overlapping_chymotrypsin_seqs = set.intersection(set(original_chymotrypsin_seqs), set(pipeline_chymotrypsin_seqs))
print("Number of sequences that overlap is:")
print("{0} for trypsin".format(len(overlapping_trypsin_seqs)))
print("{0} for chymotrypsin".format(len(overlapping_chymotrypsin_seqs)))

In [ ]:
for protease in proteases:
    print("\n------------------------------------------")
    print("Correlation plots for {0}".format(protease))

    # Define sequences that are shared between the two replicates
    seqs1 = counts_dfs['original'][protease].index.values
    seqs2 = counts_dfs['pipeline'][protease].index.values
    shared_seqs = set.intersection(set(seqs1), set(seqs2))
    print("Number of overlapping sequences is: {0}".format(len(shared_seqs)))

    x = counts_dfs['original'][protease].loc[shared_seqs]['counts0']
    y = counts_dfs['pipeline'][protease].loc[shared_seqs]['counts0']
    g = sns.jointplot(x=x, y=y)

    # Make the plot square with the same ranges, add title, and fix axis labels
    max_freq = max(x + y)
    g.set_axis_labels(xlabel='original', ylabel='pipeline')

    # Add text with correlation coefficient to plots
    (r,p) = scipy.stats.pearsonr(x, y)
    print('R = {0}'.format(r))
    #plt.text(0.05, 0.75, '$R$ = {0}'.format(round(r, 2)), transform=ax.transAxes)

    plt.axis('equal')
    plt.tight_layout()
    plt.show()

Find sequences that are present in the original data, but absent in data from the pipeline.

In [ ]:
counts_dfs['original'].keys()

In [ ]:
missing_trypsin_seqs = set(original_trypsin_seqs).difference(set(pipeline_trypsin_seqs))
counts_dfs['original']['trypsin'].loc[missing_trypsin_seqs]

The below cells are for testing parts of the script in this notebook

In [ ]:
# Import `Python` modules
import os
import glob

import pandas

In [ ]:
# Read in the designed sequences
designed_seqs_df = pandas.read_csv(designed_sequences_file)
designed_seqs_df.set_index('protein_sequence', inplace=True)

# Read in data from the experiments summary file and get a list of
# unique proteases and unique selection indices
summary_df = pandas.read_csv(experimental_summary_file)
proteases = set(summary_df['protease_type'])
selection_indices = set(summary_df['selection_strength'])
print(proteases)
print(selection_indices)

In [ ]:
# Iterate through each sample in the experimental summary dataframe and compile
# a list of FASTQ files for each sample
FASTQ_files = {}
for (i, row) in summary_df.iterrows():

    # Get sample metadata
    protease_type = row['protease_type']
    selection_index = row['selection_strength']
    experiment_name = '{0}_{1}'.format(protease_type, selection_index)
    fastq_id = row['fastq_id'].replace('_', '-')

    # Find R1 and R2 files and append them to a list
    r1_files = glob.glob('{0}/{1}*_R1_*.fastq*'.format(fastq_dir, fastq_id))
    r2_files = [f.replace('_R1_', '_R2_') for f in r1_files]
    assert experiment_name not in FASTQ_files.keys(), \
        "Duplicate experiment name: {0}".format(experiment_name)
    FASTQ_files[experiment_name] = list(zip(r1_files, r2_files))

    # Make sure that each sample has the same number of R1 and R2 files, that
    # there are more than one of each, and that the patterns "_R1_" and "_R2_"
    # don't appear more than once
    assert(len(r1_files) == len(r2_files))
    if len(r1_files) == 0:
        raise ValueError(
            "Failed to find FASTQ files for the fastq_id: {0}".format(fastq_ID)
        )
    for (f1, f2) in zip(r1_files, r2_files):
        assert f1.count('_R1_') == f2.count('_R2_') == 1, \
            "The string '_R1_' or '_R2_' appear multiple times in file name"
        if not os.path.isfile(f2):
            raise ValueError(
                "Failed to find a matching R2 file for: {0}".format(f)
            )
